In [7]:
import cv2
import mediapipe as mp
import numpy as np
import time

# Initialize Mediapipe utilities
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [8]:
for lndmrk in mp_pose.PoseLandmark:
    print(lndmrk)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [9]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 
    

In [10]:
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

In [11]:
shoulder, elbow, wrist

([0.6606784462928772, 0.8971905708312988],
 [0.7400370836257935, 1.2955639362335205],
 [0.5835765600204468, 1.0196843147277832])

In [12]:
calculate_angle(shoulder, elbow, wrist)

18.292795951262512

In [13]:
tuple(np.multiply(elbow, [640, 480]).astype(int))

(473, 621)

: 

In [20]:
cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0 
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Curl counter logic
            if angle > 160:
                stage = " down"
            if angle < 30 and stage ==' down':
                stage=" up"
                counter +=1
                print(counter)
                       
        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

I0000 00:00:1739308431.074307  460077 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2 Pro
W0000 00:00:1739308431.145935  462980 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1739308431.157910  462980 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26


: 

In [13]:
engine = pyttsx3.init()

In [14]:
exercises = {
    "bicep_curl": {"joints": ["shoulder", "elbow", "wrist"], "min_angle": 30, "max_angle": 160},
    "squat": {"joints": ["hip", "knee", "ankle"], "min_angle": 90, "max_angle": 170},
    "push_up": {"joints": ["shoulder", "elbow", "wrist"], "min_angle": 70, "max_angle": 160}
}

In [15]:
exercise_name = input("Enter exercise (bicep_curl, squat, push_up): ").strip().lower()
if exercise_name not in exercises:
    print("Invalid exercise. Defaulting to bicep_curl.")
    exercise_name = "bicep_curl"
exercise = exercises[exercise_name]



In [16]:
# Video capture
cap = cv2.VideoCapture(0)

In [17]:
# Exercise Tracking Variables
counter = 0
stage = None
last_feedback = ""
start_time = time.time()
rep_times = []  # Track time per rep
angle_ranges = []  # Track range of motion


In [18]:
def estimate_calories(reps):
    return reps * 0.2  # Adjust based on exercise type

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Initialize video capture
cap = cv2.VideoCapture(0)

# Exercise settings (customize as needed)
exercise = {
    "joints": ["ELBOW", "SHOULDER", "WRIST"],  # Example: Adjust as per exercise
    "min_angle": 30,  # Minimum angle for rep counting
    "max_angle": 160  # Maximum angle for rep counting
}

# Function to calculate angles between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First joint
    b = np.array(b)  # Mid joint
    c = np.array(c)  # Last joint

    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)

# Function to estimate calories burned (simplified)
def estimate_calories(reps):
    return reps * 0.5  # Example: Adjust formula as needed

# Tracking variables
counter = 0
stage = None
rep_times = []
angle_ranges = []
last_feedback = ""
start_time = time.time()

# Pose detection
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to read frame. Exiting...")
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            
            # Extract joint coordinates dynamically
            joint_names = exercise["joints"]
            joint_coords = []
            try:
                for joint in joint_names:
                    landmark_index = getattr(mp_pose.PoseLandmark, f"LEFT_{joint.upper()}").value
                    joint_coords.append([landmarks[landmark_index].x, landmarks[landmark_index].y])
                
                # Calculate angle
                angle = calculate_angle(*joint_coords)
                
                # Rep count logic
                if angle < exercise["min_angle"] and stage == 'down':
                    stage = 'up'
                if angle > exercise["max_angle"] and stage == 'up':
                    stage = 'down'
                    counter += 1
                    rep_times.append(time.time())
                    angle_ranges.append(angle)
                
                # Fatigue Detection
                if len(rep_times) > 5:
                    avg_rep_time = np.mean(np.diff(rep_times[-5:]))  
                    motion_range = np.mean(angle_ranges[-5:])  
                
                # Display rep count, workout time, and calorie estimation
                elapsed_time = int(time.time() - start_time)
                calories_burned = estimate_calories(counter)
                
                cv2.putText(image, f'Reps: {counter}', (10, 50), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                cv2.putText(image, f'Angle: {int(angle)}', (10, 100), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
                cv2.putText(image, f'Time: {elapsed_time}s', (10, 150), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2, cv2.LINE_AA)
                cv2.putText(image, f'Calories: {calories_burned:.1f}', (10, 200), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 165, 255), 2, cv2.LINE_AA)

                # Form correction feedback
                if angle > exercise["max_angle"]:
                    feedback = "Extend fully!"
                    color = (0, 0, 255)
                elif angle < exercise["min_angle"]:
                    feedback = "Good form!"
                    color = (0, 255, 0)
                else:
                    feedback = "Keep going!"
                    color = (255, 255, 0)
                
                if feedback != last_feedback:
                    last_feedback = feedback
                
                cv2.putText(image, feedback, (10, 250), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
            except AttributeError:
                print("Error retrieving joint coordinates. Check joint names.")

            # **Draw Pose Landmarks (Fix for missing blue lines)**
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=3),  
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2)
            )

        # Show image
        cv2.imshow('Exercise Tracker', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


2025-02-11 20:11:32.355 Python[65347:495270] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
I0000 00:00:1739322693.625136  495270 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1739322693.712308  495433 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1739322693.726890  495435 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1739322693.882972  495437 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
2025-02-11 20:11:34.155 

: 